In [14]:
import torch
import numpy as np
from torch import nn

In [3]:
layer1 = nn.Linear(8,3)

In [4]:
torch.ones((3,8))

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [5]:
layer1.weight.size(1)

8

In [6]:
layer1.weight[:,4:], layer1.weight[:,:4]

(tensor([[-0.0581,  0.0361, -0.0649,  0.0933],
         [ 0.2731, -0.2574, -0.2784, -0.2805],
         [ 0.2419,  0.2746,  0.2415,  0.3205]], grad_fn=<SliceBackward0>),
 tensor([[-0.3356, -0.2775,  0.2027,  0.0922],
         [-0.1183,  0.1392, -0.2094,  0.2146],
         [-0.0479,  0.0785,  0.1698, -0.1927]], grad_fn=<SliceBackward0>))

In [7]:
x = torch.ones((2,3))
y = torch.zeros((2,6))
z = torch.cat((x,y),dim=1)
z

tensor([[1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0.]])

In [8]:
x = torch.ones((18,20))
gama, beta = torch.split(x,10,1)
gama.shape, beta.shape

(torch.Size([18, 10]), torch.Size([18, 10]))

In [2]:
from transformers import AutoModelForSeq2SeqLM
from transformers import BertModel, BertConfig
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

bertconfig = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)

peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

model = BertModel.from_pretrained('bert-base-uncased', config=bertconfig)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
# trainable params: 294912 || all params: 109777152 || trainable%: 0.2686460658042941

D:\Conada\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing 

trainable params: 294912 || all params: 109777152 || trainable%: 0.2686460658042941


In [3]:
bertModel = BertModel.from_pretrained('bert-base-uncased', config=bertconfig)
# y = bertModel(inputs["input_ids"])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# TARGET_MODULES = [
#     "query",
#     "v_proj",
# ]

# TARGET_MODULES = "v_proj"

In [17]:
import re
key_list = [key for key, _ in bertModel.named_modules()]
for i in list(range(len(key_list))):
     # target_module_found = any(key_list[i].endswith(target_key) for target_key in TARGET_MODULES)
     # if target_module_found:
    print(key_list[i])


embeddings
embeddings.word_embeddings
embeddings.position_embeddings
embeddings.token_type_embeddings
embeddings.LayerNorm
embeddings.dropout
encoder
encoder.layer
encoder.layer.0
encoder.layer.0.attention
encoder.layer.0.attention.self
encoder.layer.0.attention.self.query
encoder.layer.0.attention.self.key
encoder.layer.0.attention.self.value
encoder.layer.0.attention.self.dropout
encoder.layer.0.attention.output
encoder.layer.0.attention.output.dense
encoder.layer.0.attention.output.LayerNorm
encoder.layer.0.attention.output.dropout
encoder.layer.0.intermediate
encoder.layer.0.intermediate.dense
encoder.layer.0.intermediate.intermediate_act_fn
encoder.layer.0.output
encoder.layer.0.output.dense
encoder.layer.0.output.LayerNorm
encoder.layer.0.output.dropout
encoder.layer.1
encoder.layer.1.attention
encoder.layer.1.attention.self
encoder.layer.1.attention.self.query
encoder.layer.1.attention.self.key
encoder.layer.1.attention.self.value
encoder.layer.1.attention.self.dropout
encoder.

In [43]:
import loralib as lora
# Add a pair of low-rank adaptation matrices with rank r=16
layer = lora.Linear(512, 512, r=16)
lora.mark_only_lora_as_trainable(bertModel)

In [33]:
import torch
print(torch.sub(input=y[1],other=x[1]))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [2]:
from transformers import BertModel, BertConfig
from models.lora import *
from config import *

config = Config()
bertconfig = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
model = BertModel.from_pretrained('bert-base-uncased', config=bertconfig)
PEFTmodel = LoraModel(config=config,model=model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


AttributeError: 'Config' object has no attribute 'inference_mode'

In [1]:
x = 'MOSEI'
print(str(x).lower())

mosei


In [2]:
x = 3.5e-6
print(x)

3.5e-06
